## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Apr. 20th Apr. 27th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,April-22-2025,Condo/Co-op,555 N Riverside Dr #12,Pompano Beach,FL,33062.0,270000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/555-N-...,Stellar MLS as Distributed by MLS Grid,GC529440,N,Y,26.240917,-80.091249


In [8]:
print('Input county name:')
county = input()

Input county name:
Broward


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [11]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-04-25    35
2025-04-21    31
2025-04-23    27
2025-04-22    25
2025-04-24    19
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
17,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465
30,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Beaches MLS,RX-11066286,N,Y,26.232119,-80.091722
113,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10471725,N,Y,26.168258,-80.099279
72,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11048992,N,Y,26.312557,-80.076756
8,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,Beaches MLS,RX-11062421,N,Y,25.984278,-80.118815
2,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10495495,N,Y,26.155455,-80.100849
54,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10487508,N,Y,26.118136,-80.107099
126,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10462721,N,Y,26.163462,-80.101526
5,PAST SALE,2025-04-23,Condo/Co-op,609 Bayshore Dr #1,Fort Lauderdale,FL,33304.0,810000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/609-...,Beaches MLS,F10493527,N,Y,26.131498,-80.109140
106,PAST SALE,2025-04-25,Condo/Co-op,4111 S Ocean Dr #3503,Hollywood,FL,33019.0,785000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11699311,N,Y,25.986806,-80.118650


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,...,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None
1,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,...,-80.091722,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Edward Cook,"Continental Properties, Inc.",None,None
2,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,...,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None
3,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,...,-80.076756,https://www.redfin.com/FL/Deerfield-Beach/2051...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Veronica Nagliatti,Fortune Christie's International Real Estate,None,None
4,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,...,-80.118815,https://www.redfin.com/FL/Hallandale-Beach/185...,Isabella Freedberg,Luxury Partners Realty,None,None,Marie Bellitto,Cervera Real Estate Inc,None,None
5,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,...,-80.100849,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Richard Salter,"Compass Florida, LLC",Christopher MacInnis,"Compass Florida, LLC",None,Coldwell Banker Realty,None,None
6,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,...,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Christian Duhaime,RE/MAX Consultants Realty 1,None,None,Ester Kadosh,Coldwell Banker Realty,None,None
7,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,...,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Robert Lopardo,Coldwell Banker Realty,None,None,Linda Gentile,Lauderdale Beach Properties,Romina Oliviero,Lauderdale Beach Properties
8,PAST SALE,2025-04-23,Condo/Co-op,609 Bayshore Dr #1,Fort Lauderdale,FL,33304.0,810000.0,2.0,2.0,...,-80.109140,https://www.redfin.com/FL/Fort-Lauderdale/609-...,Andrew Ziffer,"Compass Florida, LLC",Michelle Akerstrom,"Compass Florida, LLC",Andrew Ziffer,"Compass Florida, LLC",Michelle Akerstrom,"Compass Florida, LLC"
9,PAST SALE,2025-04-25,Condo/Co-op,4111 S Ocean Dr #3503,Hollywood,FL,33019.0,785000.0,2.0,2.0,...,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Ryan Mendell,Maxwell E Realty Inc,Oren Cytrynbaum,Maxwell E Realty Inc,Graciela Kaminsky,"Aspen International Realty,Inc",None,None


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

137


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$336,236


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$272


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$46,064,280


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          137              336236          272                     46064280.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
117
-----------
Input Previous Week Condo Average Sales Price:
436635
-----------
Input Previous Week Condo Average PSF:
316
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
51086250.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange
1,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,Plaza AT Oceanside,2726.0,NaN,2008.0,NaN,954.0,2186.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Beaches MLS,RX-11066286,N,Y,26.232119,-80.091722,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Edward Cook,"Continental Properties, Inc.",None,None,2,blue
2,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,L'hermitage,1977.0,NaN,1997.0,NaN,885.0,2765.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10471725,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,3,blue
3,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,Ocean Plaza ON Deerfield Beach,3021.0,NaN,2003.0,NaN,563.0,2091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11048992,N,Y,26.312557,-80.076756,https://www.redfin.com/FL/Deerfield-Beach/2051...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Veronica Nagliatti,Fortune Christie's International Real Estate,None,None,4,blue
4,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,Beach Club Tower 1,2078.0,NaN,2005.0,NaN,816.0,2355.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,Beaches MLS,RX-11062421,N,Y,25.984278,-80.118815,https://www.redfin.com/FL/Hallandale-Beach/185...,Isabella Freedberg,Luxury Partners Realty,None,None,Marie Bellitto,Cervera Real Estate Inc,None,None,5,blue
5,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,The Palms,1941.0,NaN,1999.0,NaN,577.0,2831.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10495495,N,Y,26.155455,-80.100849,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Richard Salter,"Compass Florida, LLC",Christopher MacInnis,"Compass Florida, LLC",None,Coldwell Banker Realty,None,None,6,blue
6,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1970.0,NaN,550.0,1285.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10487508,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Christian Duhaime,RE/MAX Consultants Realty 1,None,None,Ester Kadosh,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,Sapphire,1274.0,NaN,2009.0,NaN,640.0,1397.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10462721,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Robert Lopardo,Coldwell Banker Realty,None,None,Linda Gentile,Lauderdale Beach Properties,Romina Oliviero,Lauderdale Beach Properties,8,blue
8,PAST SALE,2025-04-23,Condo/Co-op,609 Bayshore Dr #1,Fort Lauderdal

In [35]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [36]:
m.save('index.html')

## Data snagger

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [38]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_04282025


# CREATE TEMPLATE 

In [40]:
muni_set = set(df_top_ten['CITY'])

In [41]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [42]:
df_top_ten.reset_index(inplace=True,drop=True)

In [43]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [44]:
top_sale

"L'hermitage at 3200 N Ocean Blvd #609 in Fort Lauderdale"

In [45]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange,L'hermitage at 3200 N Ocean Blvd #609 in Fort ...
1,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,Plaza AT Oceanside,2726.0,NaN,2008.0,NaN,954.0,2186.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Beaches MLS,RX-11066286,N,Y,26.232119,-80.091722,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Edward Cook,"Continental Properties, Inc.",None,None,2,blue,Plaza AT Oceanside at 1 N Ocean Blvd #1607 in ...
2,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,L'hermitage,1977.0,NaN,1997.0,NaN,885.0,2765.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10471725,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,3,blue,L'hermitage at 3100 N Ocean Blvd #1404 in Fort...
3,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,Ocean Plaza ON Deerfield Beach,3021.0,NaN,2003.0,NaN,563.0,2091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11048992,N,Y,26.312557,-80.076756,https://www.redfin.com/FL/Deerfield-Beach/2051...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Veronica Nagliatti,Fortune Christie's International Real Estate,None,None,4,blue,Ocean Plaza ON Deerfield Beach at 2051 SE 3rd ...
4,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,Beach Club Tower 1,2078.0,NaN,2005.0,NaN,816.0,2355.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,Beaches MLS,RX-11062421,N,Y,25.984278,-80.118815,https://www.redfin.com/FL/Hallandale-Beach/185...,Isabella Freedberg,Luxury Partners Realty,None,None,Marie Bellitto,Cervera Real Estate Inc,None,None,5,blue,Beach Club Tower 1 at 1850 S Ocean Dr #2304 in...
5,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,The Palms,1941.0,NaN,1999.0,NaN,577.0,2831.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10495495,N,Y,26.155455,-80.100849,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Richard Salter,"Compass Florida, LLC",Christopher MacInnis,"Compass Florida, LLC",None,Coldwell Banker Realty,None,None,6,blue,The Palms at 2100 N Ocean Blvd Unit 8A in Fort...
6,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1970.0,NaN,550.0,1285.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10487508,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Christian Duhaime,RE/MAX Consultants Realty 1,None,None,Ester Kadosh,Coldwell Banker Realty,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1210 i...
7,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,Sapphire,1274.0,NaN,2009.0,NaN,640.0,1397.0,Sold,NaN,

In [48]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: L'hermitage closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $785,000 to $3,300,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Deerfield Beach, Hallandale Beach, Hollywood, Pompano Beach, Fort Lauderdale, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 137 condo sales totaling $46,064,280 million from Apr. 20th to Apr. 27th. The previous week, brokers closed 117 condo sales totaling $51,086,250.

Last week’s units sold for an average of $336,236, lower than the $436,635 average price from the pre

In [49]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [50]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange,L'hermitage at 3200 N Ocean Blvd #609 in Fort ...
1,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,Plaza AT Oceanside,2726.0,NaN,2008.0,NaN,954.0,2186.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Beaches MLS,RX-11066286,N,Y,26.232119,-80.091722,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Edward Cook,"Continental Properties, Inc.",None,None,2,blue,Plaza AT Oceanside at 1 N Ocean Blvd #1607 in ...
2,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,L'hermitage,1977.0,NaN,1997.0,NaN,885.0,2765.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10471725,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,3,blue,L'hermitage at 3100 N Ocean Blvd #1404 in Fort...
3,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,Ocean Plaza ON Deerfield Beach,3021.0,NaN,2003.0,NaN,563.0,2091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11048992,N,Y,26.312557,-80.076756,https://www.redfin.com/FL/Deerfield-Beach/2051...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Veronica Nagliatti,Fortune Christie's International Real Estate,None,None,4,blue,Ocean Plaza ON Deerfield Beach at 2051 SE 3rd ...
4,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,Beach Club Tower 1,2078.0,NaN,2005.0,NaN,816.0,2355.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,Beaches MLS,RX-11062421,N,Y,25.984278,-80.118815,https://www.redfin.com/FL/Hallandale-Beach/185...,Isabella Freedberg,Luxury Partners Realty,None,None,Marie Bellitto,Cervera Real Estate Inc,None,None,5,blue,Beach Club Tower 1 at 1850 S Ocean Dr #2304 in...
5,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,The Palms,1941.0,NaN,1999.0,NaN,577.0,2831.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10495495,N,Y,26.155455,-80.100849,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Richard Salter,"Compass Florida, LLC",Christopher MacInnis,"Compass Florida, LLC",None,Coldwell Banker Realty,None,None,6,blue,The Palms at 2100 N Ocean Blvd Unit 8A in Fort...
6,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1970.0,NaN,550.0,1285.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10487508,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Christian Duhaime,RE/MAX Consultants Realty 1,None,None,Ester Kadosh,Coldwell Banker Realty,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1210 i...
7,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,Sapphire,1274.0,NaN,2009.0,NaN,640.0,1397.0,Sold,NaN,

In [53]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: L'hermitage closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $785,000 to $3,300,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Deerfield Beach, Hallandale Beach, Hollywood, Pompano Beach, Fort Lauderdale, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 137 condo sales totaling $46,064,280 million from Apr. 20th to Apr. 27th. The previous week, brokers closed 117 condo sales totaling $51,086,250.

Last week’s units sold for an average of $336,236, lower than the $436,635 average price from the pre

In [54]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/3200-N-Ocean-Blvd-33308/unit-609/home/41972788


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Pompano-Beach/1-N-Ocean-Blvd-33062/unit-1607/home/42070652


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hollywood/4111-S-Ocean-Dr-33019/unit-3503/home/56678133


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/609-Bayshore-Dr-33304/unit-1/home/41978343


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/3200-N-Ocean-Blvd-33308/unit-609/home/41972788


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Fort-Lauderdale/1-Las-Olas-Cir-33316/unit-1210/home/41801735


In [60]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-24,Condo/Co-op,3200 N Ocean Blvd #609,Fort Lauderdale,FL,33308.0,3300000.0,3.0,4.0,L'hermitage,2850.0,NaN,1998.0,NaN,1158.0,3956.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10495926,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange,L'hermitage at 3200 N Ocean Blvd #609 in Fort ...
1,PAST SALE,2025-04-25,Condo/Co-op,1 N Ocean Blvd #1607,Pompano Beach,FL,33062.0,2600000.0,3.0,3.5,Plaza AT Oceanside,2726.0,NaN,2008.0,NaN,954.0,2186.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Beaches MLS,RX-11066286,N,Y,26.232119,-80.091722,https://www.redfin.com/FL/Pompano-Beach/1-N-Oc...,Bonnie Heatzig,Compass Florida LLC,None,None,Edward Cook,"Continental Properties, Inc.",None,None,2,blue,Plaza AT Oceanside at 1 N Ocean Blvd #1607 in ...
2,PAST SALE,2025-04-21,Condo/Co-op,3100 N Ocean Blvd #1404,Fort Lauderdale,FL,33308.0,1750000.0,3.0,2.5,L'hermitage,1977.0,NaN,1997.0,NaN,885.0,2765.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10471725,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,3,blue,L'hermitage at 3100 N Ocean Blvd #1404 in Fort...
3,PAST SALE,2025-04-25,Condo/Co-op,2051 SE 3rd St Ph 3,Deerfield Beach,FL,33441.0,1700000.0,4.0,4.5,Ocean Plaza ON Deerfield Beach,3021.0,NaN,2003.0,NaN,563.0,2091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11048992,N,Y,26.312557,-80.076756,https://www.redfin.com/FL/Deerfield-Beach/2051...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Veronica Nagliatti,Fortune Christie's International Real Estate,None,None,4,blue,Ocean Plaza ON Deerfield Beach at 2051 SE 3rd ...
4,PAST SALE,2025-04-21,Condo/Co-op,1850 S Ocean Dr #2304,Hallandale Beach,FL,33009.0,1695000.0,3.0,3.5,Beach Club Tower 1,2078.0,NaN,2005.0,NaN,816.0,2355.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,Beaches MLS,RX-11062421,N,Y,25.984278,-80.118815,https://www.redfin.com/FL/Hallandale-Beach/185...,Isabella Freedberg,Luxury Partners Realty,None,None,Marie Bellitto,Cervera Real Estate Inc,None,None,5,blue,Beach Club Tower 1 at 1850 S Ocean Dr #2304 in...
5,PAST SALE,2025-04-25,Condo/Co-op,2100 N Ocean Blvd Unit 8A,Fort Lauderdale,FL,33305.0,1120000.0,3.0,3.5,The Palms,1941.0,NaN,1999.0,NaN,577.0,2831.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10495495,N,Y,26.155455,-80.100849,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Richard Salter,"Compass Florida, LLC",Christopher MacInnis,"Compass Florida, LLC",None,Coldwell Banker Realty,None,None,6,blue,The Palms at 2100 N Ocean Blvd Unit 8A in Fort...
6,PAST SALE,2025-04-21,Condo/Co-op,1 Las Olas Cir #1210,Fort Lauderdale,FL,33316.0,825000.0,2.0,2.0,Venetian Condominium,1500.0,NaN,1970.0,NaN,550.0,1285.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Beaches MLS,F10487508,N,Y,26.118136,-80.107099,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Christian Duhaime,RE/MAX Consultants Realty 1,None,None,Ester Kadosh,Coldwell Banker Realty,None,None,7,blue,Venetian Condominium at 1 Las Olas Cir #1210 i...
7,PAST SALE,2025-04-23,Condo/Co-op,2821 N Ocean Blvd Unit 506S,Fort Lauderdale,FL,33308.0,815000.0,2.0,2.0,Sapphire,1274.0,NaN,2009.0,NaN,640.0,1397.0,Sold,NaN,

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 2, 27) ## List (Earlier) date
date2 = datetime(2025, 4, 25) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

57


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['$/SQUARE FEET'].astype(float)

0    1158.0
1     954.0
2     885.0
3     563.0
4     816.0
5     577.0
6     550.0
7     640.0
8     736.0
9     636.0
Name: $/SQUARE FEET, dtype: float64

In [70]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [72]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [73]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-04-24,3200 N Ocean Blvd #609 in Fort Lauderdale,"$3,300,000",3,4.0,"2,850",1998,"$1,158",Beaches MLS,F10495926,"Patricia Post Post and Post, Inc.","Patricia Post Post and Post, Inc."
1,2025-04-25,1 N Ocean Blvd #1607 in Pompano Beach,"$2,600,000",3,3.5,"2,726",2008,$954,Beaches MLS,RX-11066286,Bonnie Heatzig Compass Florida LLC,"Edward Cook Continental Properties, Inc."
2,2025-04-21,3100 N Ocean Blvd #1404 in Fort Lauderdale,"$1,750,000",3,2.5,"1,977",1997,$885,Beaches MLS,F10471725,"Patricia Post Post and Post, Inc.","Patricia Post Post and Post, Inc."
3,2025-04-25,2051 SE 3rd St Ph 3 in Deerfield Beach,"$1,700,000",4,4.5,"3,021",2003,$563,Beaches MLS,RX-11048992,Michael Wells Premier Estate Properties Inc Sc...,Veronica Nagliatti Fortune Christie's Internat...
4,2025-04-21,1850 S Ocean Dr #2304 in Hallandale Beach,"$1,695,000",3,3.5,"2,078",2005,$816,Beaches MLS,RX-11062421,Isabella Freedberg Luxury Partners Realty,Marie Bellitto Cervera Real Estate Inc
5,2025-04-25,2100 N Ocean Blvd Unit 8A in Fort Lauderdale,"$1,120,000",3,3.5,"1,941",1999,$577,Beaches MLS,F10495495,"Richard Salter Compass Florida, LLC Christophe...",Coldwell Banker Realty
6,2025-04-21,1 Las Olas Cir #1210 in Fort Lauderdale,"$825,000",2,2.0,"1,500",1970,$550,Beaches MLS,F10487508,Christian Duhaime RE/MAX Consultants Realty 1,Ester Kadosh Coldwell Banker Realty
7,2025-04-23,2821 N Ocean Blvd Unit 506S in Fort Lauderdale,"$815,000",2,2.0,"1,274",2009,$640,Beaches MLS,F10462721,Robert Lopardo Coldwell Banker Realty,Linda Gentile Lauderdale Beach Properties Romi...
8,2025-04-23,609 Bayshore Dr #1 in Fort Lauderdale,"$810,000",2,2.0,"1,100",1958,$736,Beaches MLS,F10493527,"Andrew Ziffer Compass Florida, LLC Michelle Ak...","Andrew Ziffer Compass Florida, LLC Michelle Ak..."
9,2025-04-25,4111 S Ocean Dr #3503 in Hollywood,"$785,000",2,2.0,"1,235",2017,$636,MARMLS,A11699311,Ryan Mendell Maxwell E Realty Inc Oren Cytrynb...,"Graciela Kaminsky Aspen International Realty,I..."
